
<a href="https://colab.research.google.com/github/sndnyang/iDDPM/blob/master/iddpm_gen_fid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iDDPM Generate and FID

Generating/sampling new images and evaluate FID


In [ ]:
# Download the pre-trained model from openAI iDDPM
!wget https://openaipublic.blob.core.windows.net/diffusion/march-2021/cifar10_uncond_50M_500K.pt

: 

In [ ]:
!pip install tensorboardX torch_fidelity

In [ ]:

# If you are use colab, you can mount your Google Drive.
# Otherwise, you need to upload the code?
import sys

sys.path.append('/drive/MyDrive/research/iDDPM')

In [ ]:

import argparse
import numpy as np
import torch.optim
import torch.utils.data
import torch.nn as nn
import torchvision.transforms as transforms

from ExpUtils import *
from utils.dataloader import datainfo, dataload
from eval_tasks import *
from iddpm.script_util import create_model_and_diffusion, model_and_diffusion_defaults

In [ ]:

arg = argparse.Namespace()
arg.dataset = 'cifar10'
arg.data_path = './data'
arg.workers = 4
arg.eval = 'nll'
arg.resume = 'cifar10_uncond_50M_500K.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
arg.device = device

: 

In [ ]:

data_info = datainfo(logger, arg)

if arg.eval == 'buffer':
    buffer = torch.load(arg.resume, map_location=arg.device)
    eval_buffer(buffer, arg)

normalize = [transforms.Normalize(mean=data_info['stat'][0], std=data_info['stat'][1])]
augmentations = transforms.Compose([
    transforms.ToTensor(),
    *normalize
])

'''
    model
'''

model, diffusion = create_model_and_diffusion(
    **model_and_diffusion_defaults()
)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of params: {format(n_parameters, ",")}')


In [ ]:
# new_samples(model, diffusion, arg) in eval_tasks.py

im_sz = arg.image_size
start = time.time()
batches = 100 if arg.dataset != 'stl10' else 50
replay_buffer = torch.FloatTensor(batches * 100, n_ch, im_sz, im_sz).uniform_(-1, 1)
eval_time = 0
bs = 100

# sample_fn = (
#     diffusion.p_sample_loop if not arg.use_ddim else diffusion.ddim_sample_loop
# )

for i in range(batches):

    samples = diffusion.p_sample_loop(
        model, (bs, 3, im_sz, im_sz), clip_denoised=True, model_kwargs=None
    )
    replay_buffer[i * bs: (i + 1) * bs] = (samples + 1) / 2

    if (i + 1) % 10 == 0:
        now = time.time()
        print(f'batch {i + 1} {str(timedelta(seconds=now - start - eval_time))}')

    if (i + 1) in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500]:
        eval_start = time.time()
        plot('{}/samples_{}.png'.format(arg.dir_path, i+1), samples)
        metrics = eval_is_fid(replay_buffer[:(i + 1) * bs] * 255, dataset=arg.dataset, args=arg)
        inc_score = metrics['inception_score_mean']
        fid = metrics['frechet_inception_distance']
        print("sample with %d" % (i * bs + bs))
        print("Inception score of {}".format(inc_score))
        print("FID of score {}".format(fid))
        eval_time += time.time() - eval_start